In [1]:
#Box Office and Budget Adjusted
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Load the Excel file
df = pd.read_excel('superhero_box_office.xlsx', parse_dates=['Release Date'])

# CPI data (approximate U.S. annual averages, base 2025 = 120.0)
cpi_data = {1989:46.5,1990:49.0,1991:51.1,1992:52.7,1993:54.2,1994:55.6,1995:57.2,1996:58.9,1997:60.2,1998:61.2,1999:62.5,2000:64.6,2001:66.4,
            2002:67.5,2003:69.0,2004:70.9,2005:73.3,2006:75.6,2007:77.8,2008:80.8,2009:80.5,2010:81.8,2011:84.4,2012:86.1,2013:87.4,2014:88.8,
            2015:88.9,2016:90.0,2017:92.0,2018:94.2,2019:95.9,2020:97.1,2021:101.7,2022:109.8,2023:114.4,2024:117.7,2025:120.0}



# Function to get CPI for a release year
def get_cpi(year):
    return cpi_data.get(year, cpi_data[2025])

# Adjust grosses for inflation (to 2025 dollars, in millions)
cpi_2025 = cpi_data[2025]
df['Year'] = df['Release Date'].dt.year
df['Adjusted Domestic (Millions)'] = (df['Domestic Gross'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted International (Millions)'] = (df['International Gross'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Worldwide (Millions)'] = (df['Worldwide Gross'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Production Budget (Millions)'] = (df['Production Budget'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Marketing Budget (Millions)'] = (df['Marketing Budget'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Opening Weekend (Millions)'] = (df['Opening Weekend'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Domestic Revenue (Millions)'] = (df['Domestic Revenue'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted International Revenue (Millions)'] = (df['International Revenue'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Adjusted Worldwide Revenue (Millions)'] = (df['Worldwide Revenue'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Opening Weekend Percentage of Total Gross'] = df['Percent of Total Gross']
df['Movie Legs'] = df['Legs']
df['Domestic Box Office Share'] = df['Domestic Share Percentage']
df['International Box Office Share'] = (100 - df['Domestic Share Percentage'])

# Sort by Release Date
df = df.sort_values('Release Date')

# Create the figure
fig = go.Figure()

# Unique franchises and series
franchises = ['MCU', 'DC', 'FOX', 'SONY']
unique_series = {franchise: sorted(df[df['Franchise'] == franchise]['Series'].unique()) for franchise in franchises}

# Chart Styling
colors = {
    'MCU': ('red', 'red', 'red'),
    'DC': ('blue', 'blue', 'blue'),
    'SONY': ('black', 'black', 'black'),
    'FOX': ('yellow', 'yellow', 'yellow')
}
markers = {'MCU': 'circle', 'DC': 'square', 'SONY': 'triangle-up', 'FOX': 'diamond'}

# Add franchise traces
franchise_traces = {}
for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_dom, color_int, color_wor = colors.get(franchise, ('grey', 'lightgrey', 'darkgrey'))
    marker = markers.get(franchise, 'circle')

    # Initialize franchise traces
    franchise_traces[franchise] = []
    
    # Domestic
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Domestic (Millions)'],
        mode='lines+markers',
        name=f'{franchise} Domestic',
        line=dict(color=color_dom, dash='dot'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Domestic (Millions)'].round(2).astype(str) + 'M',
        visible=False))
    
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # International
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted International (Millions)'],
        mode='lines+markers',
        name=f'{franchise} International',
        line=dict(color=color_int, dash='dash'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted International (Millions)'].round(2).astype(str) + 'M',
        visible=False))
    
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Worldwide
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Worldwide (Millions)'],
        mode='lines+markers',
        name=f'{franchise} Worldwide',
        line=dict(color=color_wor),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Worldwide (Millions)'].round(2).astype(str) + 'M',
        visible=True))
    
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Production Budget
    fig.add_trace(go.Bar(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Production Budget (Millions)'],
        name=f'{franchise} Production Budget',
        marker=dict(color='purple'),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Production Budget (Millions)'].round(2).astype(str) + 'M',
        visible=False))
    
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Marketing Budget
    fig.add_trace(go.Bar(
        x=df_franchise['Release Date'],
        y=df_franchise['Adjusted Marketing Budget (Millions)'],
        name=f'{franchise} Marketing Budget',
        marker=dict(color='orange'),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Adjusted Marketing Budget (Millions)'].round(2).astype(str) + 'M',
        visible=False))
    
    franchise_traces[franchise].append(len(fig.data) - 1)

# Initialize Series traces
series_traces = {franchise: {} for franchise in franchises}
series_traces_start_index = len(fig.data)

# Add Series traces
for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_dom, color_int, color_wor = colors.get(franchise, ('grey', 'lightgrey', 'darkgrey'))
    marker = markers.get(franchise, 'circle')
    
    for series in unique_series[franchise]:
        df_series = df_franchise[df_franchise['Series'] == series]
        if not df_series.empty:
            series_traces[franchise][series] = []
            
            # Series Domestic
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Adjusted Domestic (Millions)'],
                mode='lines+markers',
                name=f'{series} Domestic',
                line=dict(color=color_dom, dash='dot'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Domestic (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series International
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Adjusted International (Millions)'],
                mode='lines+markers',
                name=f'{series} International',
                line=dict(color=color_int, dash='dash'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted International (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Worldwide
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Adjusted Worldwide (Millions)'],
                mode='lines+markers',
                name=f'{series} Worldwide',
                line=dict(color=color_wor),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Worldwide (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Production Budget
            fig.add_trace(go.Bar(
                x=df_series['Release Date'],
                y=df_series['Adjusted Production Budget (Millions)'],
                name=f'{franchise} {series} Production Budget',
                marker=dict(color='purple'),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Production Budget (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Marketing Budget
            fig.add_trace(go.Bar(
                x=df_series['Release Date'],
                y=df_series['Adjusted Marketing Budget (Millions)'],
                name=f'{franchise} {series} Marketing Budget',
                marker=dict(color='orange'),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Adjusted Marketing Budget (Millions)'].round(2).astype(str) + 'M',
                visible=False))
            
            series_traces[franchise][series].append(len(fig.data) - 1)

# Create franchise dropdown
franchise_dropdown = [
    
    # ALL: Show only Worldwide Box Office franchise lines
    dict(label='ALL',
        method='update',
        args=[{'visible': [True if i in [2, 7, 12, 17] else False for i in range(len(franchises) * 5)] + [False] * (len(fig.data) - len(franchises) * 5)},
        {'updatemenus[1].buttons': 
        [dict(label='None',
        method='update',
        args=[{'visible': [True if i in [2, 7, 12, 17] else False for i in range(len(franchises) * 5)] + [False] * (len(fig.data) - len(franchises) * 5)}])],
        'updatemenus[1].visible': False  # Hide series dropdown when ALL selected
        }]
        ),

    # MCU
    dict(label='MCU',
        method='update',
        args=[
        {'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]},
        {'updatemenus[1].buttons': [
        dict(label='ALL',
        method='update',
        args=[{'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]}])
        ] + [
        dict(label=series,
        method='update',
        args=[{'visible': [True if i in series_traces['MCU'][series] else False for i in range(len(fig.data))]}]
        ) for series in unique_series['MCU']],
        'updatemenus[1].visible': True
        }]
        ),

    # DC
    dict(label='DC',
        method='update',
        args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]},
        {'updatemenus[1].buttons': [
        dict(label='ALL',
        method='update',
        args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]}])
        ] + [
        dict(label=series,
        method='update',
        args=[{'visible': [True if i in series_traces['DC'][series] else False for i in range(len(fig.data))]}]
        ) for series in unique_series['DC']],
        'updatemenus[1].visible': True
        }]
        ),

    # FOX
    dict(label='FOX',
        method='update',
        args=[
        {'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]},
        {'updatemenus[1].buttons': [
        dict(label='ALL',
        method='update',
        args=[{'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]}])
        ] + [
        dict(label=series,
        method='update',
        args=[{'visible': [True if i in series_traces['FOX'][series] else False for i in range(len(fig.data))]}]
        ) for series in unique_series['FOX']],
        'updatemenus[1].visible': True
        }]
        ),
    
    # SONY
    dict(label='SONY',
        method='update',
        args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]},
        {'updatemenus[1].buttons': [            
        dict(label='ALL',
        method='update',
        args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]}])
        ] + [
        dict(label=series,
        method='update',
        args=[{'visible': [True if i in series_traces['SONY'][series] else False for i in range(len(fig.data))]}]
        ) for series in unique_series['SONY']],
        'updatemenus[1].visible': True
        }]
        )
]

# Default series dropdown (empty initially)
series_dropdown = [
    dict(label='None',
        method='update',
        args=[{'visible': [True] * len(franchises) * 5 + [False] * (len(fig.data) - len(franchises) * 5)}])
]

# Add Layout
fig.update_layout(
    title=dict(text="Estimated Superhero Movie Box Office and Budgets (Inflation-Adjusted 2025)", font=dict(size=40), font_family='Garamond', automargin=True, yref='container', x=0.5, xanchor='center', y=.98, yanchor='top'),
    xaxis_title='Release Date',
    yaxis_title='Adjusted Amount (Millions USD)',
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), 
    barmode='stack',
    updatemenus=[
        dict(
            buttons=franchise_dropdown,
            direction='down',
            showactive=True,
            x=0.15,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0
        ),
        dict(
            buttons=series_dropdown,
            direction='down',
            showactive=True,
            x=0.2,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0,
            visible=False
        )
    ],
    xaxis=dict(range=[df['Release Date'].min() - pd.Timedelta(days=365), df['Release Date'].max() + pd.Timedelta(days=365)])
)

# Save as HTML
fig.write_html('EstimatedSuperheroMovieBoxOfficeandBudgets.html')      